In [1]:
from funlib.learn.torch.models import UNet

import numpy as np
import gunpowder as gp
import torch
import zarr
import daisy

import waterz
from watershed_helpers import watershed_from_affinities

import logging

In [2]:
logging.basicConfig(level=logging.INFO)
# constants
grow = (36, 36, 36)
voxel_size = gp.Coordinate((4, 4, 4))
dims = len(voxel_size)
input_shape = gp.Coordinate((76, 76, 76)) + grow
input_size = input_shape * voxel_size
output_shape = gp.Coordinate((36, 36, 36)) + grow
output_size = output_shape * voxel_size
context = (input_size - output_size) / 2
checkpoint = "model_checkpoint_precomputed"

In [3]:
unet = UNet(
    in_channels=1,
    num_fmaps=16,
    fmap_inc_factor=5,
    downsample_factors=[(2, 2, 2), (2, 2, 2)],
    activation="ReLU",
    voxel_size=voxel_size,
    num_heads=1,
    constant_upsample=True,
)

affinities_model = torch.nn.Sequential(
    unet,
    torch.nn.Conv3d(16, dims, (1,) * dims),
    torch.nn.Sigmoid(),
)

affinities_model.eval()

Sequential(
  (0): UNet(
    (l_conv): ModuleList(
      (0): ConvPass(
        (conv_pass): Sequential(
          (0): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
          (1): ReLU()
          (2): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
          (3): ReLU()
        )
      )
      (1): ConvPass(
        (conv_pass): Sequential(
          (0): Conv3d(16, 80, kernel_size=(3, 3, 3), stride=(1, 1, 1))
          (1): ReLU()
          (2): Conv3d(80, 80, kernel_size=(3, 3, 3), stride=(1, 1, 1))
          (3): ReLU()
        )
      )
      (2): ConvPass(
        (conv_pass): Sequential(
          (0): Conv3d(80, 400, kernel_size=(3, 3, 3), stride=(1, 1, 1))
          (1): ReLU()
          (2): Conv3d(400, 400, kernel_size=(3, 3, 3), stride=(1, 1, 1))
          (3): ReLU()
        )
      )
    )
    (l_down): ModuleList(
      (0): Downsample(
        (down): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
      )
   

In [4]:
raw = gp.ArrayKey("RAW")
predictions = gp.ArrayKey("PREDICTIONS")

reference_request = gp.BatchRequest()
reference_request.add(raw, input_size)
reference_request.add(predictions, output_size)

In [5]:
source = gp.ZarrSource(
    filename="../MB-Z1213-56.zarr",
    datasets={
        raw: f"TEST/raw",
    },
)

In [6]:
# use gunpowder to check what data is provided
with gp.build(source):
    total_input_roi = source.spec[raw].roi
# shrink this down to the expected output size
total_output_roi = total_input_roi.grow(-context, -context)

# prepare a dataset
daisy.prepare_ds(
    f"predictions.zarr",
    "volumes/predictions",
    daisy.Roi(total_output_roi.get_offset(), total_output_roi.get_shape()),
    (4, 4, 4),
    np.float32,
    write_size=output_size,
    num_channels=3,
)

INFO:daisy.datasets:Reusing existing dataset


In [7]:
pipeline = (
    source
    + gp.Normalize(raw)
    + gp.Unsqueeze([raw])
    + gp.Unsqueeze([raw])
    + gp.torch.Predict(
        model=affinities_model,
        inputs={"input": raw},
        outputs={0: predictions},
        checkpoint=checkpoint,
    )
    + gp.Squeeze([raw, predictions])
    + gp.Squeeze([raw])
    + gp.ZarrWrite(
        output_dir="./",
        output_filename=f"predictions.zarr",
        dataset_names={
            raw: "volumes/raw",
            predictions: "volumes/predictions",
        },
        dataset_dtypes={predictions: gp.ArraySpec(roi=total_output_roi)},
    )
    + gp.Scan(reference_request, num_workers=3)
)

request = gp.BatchRequest()
request[raw] = gp.ArraySpec(roi=total_input_roi)
request[predictions] = gp.ArraySpec(roi=total_output_roi)

In [8]:
with gp.build(pipeline):
    pipeline.request_batch(request)

INFO:gunpowder.nodes.scan:scanning over 125 chunks
INFO:gunpowder.torch.nodes.predict:Predicting on gpu
INFO:gunpowder.torch.nodes.predict:Predicting on gpu
INFO:gunpowder.torch.nodes.predict:Predicting on gpu
INFO:gunpowder.nodes.scan:allocating array of shape (400, 400, 400) for RAW
INFO:gunpowder.nodes.scan:allocating array of shape (3, 360, 360, 360) for PREDICTIONS
INFO:gunpowder.nodes.scan:processed chunk 1/125
INFO:gunpowder.nodes.scan:processed chunk 2/125
INFO:gunpowder.nodes.scan:processed chunk 3/125
INFO:gunpowder.nodes.scan:processed chunk 4/125
INFO:gunpowder.nodes.scan:processed chunk 5/125
INFO:gunpowder.nodes.scan:processed chunk 6/125
INFO:gunpowder.nodes.scan:processed chunk 7/125
INFO:gunpowder.nodes.scan:processed chunk 8/125
INFO:gunpowder.nodes.scan:processed chunk 9/125
INFO:gunpowder.nodes.scan:processed chunk 10/125
INFO:gunpowder.nodes.scan:processed chunk 11/125
INFO:gunpowder.nodes.scan:processed chunk 12/125
INFO:gunpowder.nodes.scan:processed chunk 13/125

In [9]:
# Post processing:
results_zarr = zarr.open("predictions.zarr", "r+")
results = np.array(results_zarr["volumes/predictions"])

threshold = 0.5

predictions_source = gp.ZarrSource(
    filename="predictions.zarr",
    datasets={
        predictions: "volumes/predictions",
    },
)
with gp.build(predictions_source):
    spec = predictions_source.spec[predictions]

fragments = watershed_from_affinities(results)[0]
thresholds = [threshold]
segmentations = waterz.agglomerate(
    affs=results.astype(np.float32),
    fragments=fragments,
    thresholds=thresholds,
)

segmentation = next(segmentations)

results_zarr["volumes/segmentation"] = segmentation
results_zarr["volumes/segmentation"].attrs["offset"] = spec.roi.get_offset()
results_zarr["volumes/segmentation"].attrs["resolution"] = spec.voxel_size

Re-using already compiled waterz version
Preparing segmentation volume...
